## Accessing the data 

In YData's platform we make available scalable connectors to a variety of different data storages. The connection to the data can be made either through the UI or also via Notebooks. Below we show how we can use a connector in the notebooks.

In [1]:
import json
from ydata.connectors import GCSConnector
from ydata.connectors.filetype import FileType
from ydata.utils.formats import read_json
from pathlib import Path

token=read_json('gcs_credentials.json')  
connector = GCSConnector('ydatasynthetic', keyfile_dict=token)
data = connector.read_file(path='gs://ydata_testdata/tabular/telco_churn/data.csv', file_type=FileType.CSV)
data = data.to_pandas()
data.to_csv('original_data.csv', index=False)

### Creation of the artifact 

In order to show the downloaded data on the platform we are going to create an artifact that will be dumped in the pipeline. 

In [2]:
cat_cols=[
    "PaymentMethod",
    "MultipleLines",
    "InternetService",
    "OnlineSecurity",
    "OnlineBackup",
    "DeviceProtection",
    "TechSupport",
    "StreamingTV",
    "StreamingMovies",
    "Contract"
]

schema = [
    {
        "name": c, 
        "type": "CATEGORY" if c in cat_cols else "NUMBER"
    } for c in data.columns
]

metadata = {
    "outputs": [
        {
            "type": "table",
            "format": "csv",
            "header": [x["name"] for x in schema],
            "source": data.to_csv(header=True,index=False),
            "storage": "inline",
        }
    ]
}

with open('mlpipeline-ui-metadata.json', 'w') as f:
    json.dump(metadata, f)